In [25]:
import cv2
import mediapipe as mp
import random

In [26]:
# MediaPipe Hands 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

In [27]:
# 원 초기 설정
num_circles = 5
circles = []
score = 0

In [28]:
def create_circle(color, radius):
    return {
        "pos": [random.randint(radius, 640-radius), random.randint(radius, 480-radius)],
        "radius": radius,
        "velocity": [random.choice([-5, 5]), random.choice([-5, 5])],
        "color": color,
        "score": 5 if color == (0, 255, 0) else 3 if color == (255, 0, 0) else -3
    }

In [29]:
# 붉은색, 푸른색, 초록색 원 생성
for _ in range(num_circles):
    circles.append(create_circle((0, 0, 255), 20))  # 붉은색 원
    circles.append(create_circle((255, 0, 0), 20))  # 푸른색 원
    circles.append(create_circle((0, 255, 0), 20))  # 초록색 원 (작은 크기)

In [30]:
# 원 이동 함수
def move_circles():
    for circle in circles:
        for i in range(2):
            circle["pos"][i] += circle["velocity"][i]
            if circle["pos"][i] - circle["radius"] <= 0 or circle["pos"][i] + circle["radius"] >= (640 if i == 0 else 480):
                circle["velocity"][i] = -circle["velocity"][i]

In [31]:
# 원 그리기 함수
def draw_circles(frame):
    for circle in circles:
        cv2.circle(frame, tuple(circle["pos"]), circle["radius"], circle["color"], -1)

In [32]:
# 손이 원을 집는지 확인하는 함수
def is_circle_grabbed(hand_landmarks, circle):
    x, y = circle["pos"]
    radius = circle["radius"]
    if hand_landmarks:
        for hand_landmark in hand_landmarks:
            thumb_tip = hand_landmark.landmark[4]
            index_tip = hand_landmark.landmark[8]
            thumb_pos = (int(thumb_tip.x * 640), int(thumb_tip.y * 480))
            index_pos = (int(index_tip.x * 640), int(index_tip.y * 480))
            distance = ((thumb_pos[0] - index_pos[0]) ** 2 + (thumb_pos[1] - index_pos[1]) ** 2) ** 0.5
            if distance < radius and x - radius < index_pos[0] < x + radius and y - radius < index_pos[1] < y + radius:
                return True
    return False

In [33]:
# 주먹 쥐는 동작 확인 함수
def is_fist(hand_landmarks):
    if hand_landmarks:
        for hand_landmark in hand_landmarks:
            thumb_tip = hand_landmark.landmark[4]
            index_tip = hand_landmark.landmark[8]
            middle_tip = hand_landmark.landmark[12]
            ring_tip = hand_landmark.landmark[16]
            pinky_tip = hand_landmark.landmark[20]

            tips = [thumb_tip, index_tip, middle_tip, ring_tip, pinky_tip]
            tip_positions = [(int(tip.x * 640), int(tip.y * 480)) for tip in tips]

            distances = [((tip_positions[0][0] - tip[0]) ** 2 + (tip_positions[0][1] - tip[1]) ** 2) ** 0.5 for tip in tip_positions[1:]]
            return all(distance < 50 for distance in distances)  # 모든 손가락 끝이 서로 가까이 모여있는지 확인
    return False

In [34]:
# 웹캠 피드 초기화
cap = cv2.VideoCapture(0)

In [35]:
# 웹캠 피드 확인
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

In [36]:
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    frame = cv2.flip(frame, 1)  # 좌우 반전
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb_frame)

    if result.multi_hand_landmarks:
        if is_fist(result.multi_hand_landmarks):
            frame[:] = (0, 0, 0)  # 화면을 검정색으로 비움
            cv2.putText(frame, "End Game!", (220, 240), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 4, cv2.LINE_AA)
            cv2.imshow('Moving Circles Game', frame)  # 화면 표시
            cv2.waitKey(2000)  # 2초간 대기
            break

        for hand_landmarks in result.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            for circle in circles[:]:
                if is_circle_grabbed(result.multi_hand_landmarks, circle):
                    score += circle["score"]
                    circles.remove(circle)
                    print(f'Score: {score}')

    move_circles()  # 원 이동
    draw_circles(frame)  # 원 그리기

    cv2.putText(frame, f'Score: {score}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.imshow('Moving Circles Game', frame)  # 화면 표시

    if cv2.waitKey(1) & 0xFF == 27:  # ESC 키로 종료
        break

Score: -3
Score: 2
Score: 5
Score: 2
Score: 7
Score: 10
Score: 13
Score: 18
Score: 21


In [37]:
cap.release()
cv2.destroyAllWindows()
hands.close()